# Model Evaluation: Quick Start Guide

TODO: Intro blurb

## Define the necessary tokens

In [ ]:
%env REPLICATE_API_KEY="api_key"

## Load Libraries

In [ ]:
# pip install replicate
%pip install unitxt
%pip install openai
%pip install litellm
%pip install diskcache
%pip install scikit-learn

from unitxt.api import evaluate, load_dataset
from unitxt.inference import CrossProviderInferenceEngine
import nest_asyncio
nest_asyncio.apply()

## Load a dataset from the Unitxt catalog

In [ ]:
# Use the Unitxt APIs to load the wnli entailment dataset using the standard template in the catalog for relation task with 2-shot in-context learning.
# We set loader_limit to 20 to limit reduce inference time.
dataset = load_dataset(
    card="cards.wnli",
    template="templates.classification.multi_class.relation.default",
    format="formats.chat_api",
    num_demos=2,
    demos_pool_size=10,
    loader_limit=20,
    split="test",
)

## Instantiate the evaluation client

We are using a CrossProviderInferenceEngine inference engine that supply api access to providers such as:
watsonx, bam, openai, azure, aws and more.
For more information, visit the :ref:`inference engines guide <inference>`

TODO: point to Replicate

In [ ]:
model = CrossProviderInferenceEngine(model="granite-3-8b-instruct", provider="watsonx")

## Generate predictions

In [ ]:
predictions = model(dataset)

## Evaluate the predictions to determine results

In [6]:
results = evaluate(predictions=predictions, data=dataset)

## Print the scores

In [ ]:
print("Global Results:")
print(results.global_scores.summary)

print("Instance Results:")
print(results.instance_scores.summary)